In [12]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer.primitives import SamplerV2
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import QAOA


In [ ]:
def server_allocation_qaoa(ngb, service, server_state, N, S, U, p=2):
    # Define QUBO matrix and offset
    Q = np.zeros((U * N, U * N))
    offset = 0
    for i in range(U):
        for j in range(N):
            if ngb[i, j] == 1:  # User-server connection
                Q[i * N + j, i * N + j] = -server_capacity[j, service[i]]
                for k in range(j + 1, N):
                    if ngb[i, k] == 1:
                        Q[i * N + j, i * N + k] = 2  # Penalty for violating capacity

    # Initialize QAOA circuit
    num_qubits = U * N
    qc = QuantumCircuit(num_qubits)

    # Define QAOA parameters
    params = np.random.rand(2 * p)

    # QAOA layers
    for i in range(p):
        for j in range(num_qubits):
            qc.rx(params[2 * i], j)
        for j in range(num_qubits - 1):
            qc.cx(j, j + 1)
        for j in range(num_qubits):
            qc.rz(params[2 * i + 1], j)

    # Measure and post-process
    sampler = SamplerV2()
    job = sampler.run([qc], shots=128)
    # Perform an ideal simulation
    result_ideal = job.result()
    count = result_ideal[0].data.meas.get_counts()

    # Classical post-processing to extract allocation
    allocation = []
    for state, count in counts.items():
        state_list = [int(bit) for bit in state]
        for i in range(U):
            for j in range(N):
                if state_list[i * N + j] == 1:
                    allocation.append((i, j))
    return allocation

# Example usage
N = 5  # Number of servers
S = 2   # Number of services
U = 10  # Number of users
ngb = np.random.randint(0, 2, (U, N))  # User-server connections
service = np.random.randint(0, S, U)  # User service requests
server_state = np.random.rand(N, S)  # Server states
server_capacity = np.random.randint(1, 10, (N, S))  # Server capacities

allocation = server_allocation_qaoa(ngb, service, server_state, N, S, U)
print("QAOA Allocation:", allocation)

c:\malik inam\QC Environment\QC_Projects\.venv\Lib\site-packages\qiskit_aer\primitives\sampler_v2.py:135: UserWarning: The 0-th pub's circuit has no output classical registers and so the result will be empty. Did you mean to add measurement instructions?
  warnings.warn(


AttributeError: 'DataBin' object has no attribute 'meas'